In [ ]:
import pymysql
import pandas as pd
import warnings

## Functions

In [13]:
def decode_df(df):
    data_types = df.dtypes.to_dict()
    for col in df.columns:
        if data_types[col] == object:
            df[col] = df[col].str.decode('utf-8')
    return df

def connect(wiki_name, cnf_path = "/home/paws/.my.cnf"):
    return pymysql.connect(host = wiki_name + '.analytics.db.svc.wikimedia.cloud',
                           read_default_file = cnf_path,
                           database = wiki_name + '_p')

def pd_query(query, wiki):
    warnings.filterwarnings('ignore')
    connection = connect(wiki)
    return decode_df(pd.read_sql(query, connection))

def query_wikis(query, wikis):
    combined_result = pd.DataFrame()
    for wiki in wikis:
        result = pd_query(query, wiki)
        result['wiki_db'] = wiki
        combined_result = pd.concat([combined_result, result])
    return combined_result
    
# source: https://github.com/wikimedia/wmfdata-python/blob/main/wmfdata/utils.py
def sql_tuple(i):

    if type(i) != list:
        i = [x for x in i]

    if len(i) == 0:
        raise ValueError("Cannot produce an SQL tuple without any items.")

    list_repr = repr(i)
    return "(" + list_repr[1:-1] + ")"

## Inputs

In [49]:
test = input('Share wikis')

Share wikis test


In [50]:
test = input('Enter timestamp')

Enter timestamp test


In [51]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTPjib18Wg8gG8dD992D5QXE2R0qXTd_qaQpQQeTsfgi5DRJ6mCrgIE6PefkS6GBP17A7D7Z96cVTkw/pub?gid=1500369995&single=true&output=tsv', sep='\t')
df.head()

,revision_id,timestamp,wiki,article_title,diff,username,bytes_added,references_added,new_article,dashboard_edit
0,659472428,2022-05-28 08:00:24 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,402,0,True,False
1,659472431,2022-05-28 08:00:25 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,240,0,False,False
2,659472583,2022-05-28 08:01:24 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,429,0,False,False
3,659475427,2022-05-28 08:20:47 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,4,0,False,False
4,659476008,2022-05-28 08:25:16 UTC,https://commons.wikimedia.org,File:Residential Housing at Uganda Christian U...,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,63,0,False,False


In [41]:
df['domain_name'] = df['wiki'].str.replace('https://', '')
df.head()

,revision_id,timestamp,wiki,article_title,diff,username,bytes_added,references_added,new_article,dashboard_edit,domain_name
0,659472428,2022-05-28 08:00:24 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,402,0,True,False,commons.wikimedia.org
1,659472431,2022-05-28 08:00:25 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,240,0,False,False,commons.wikimedia.org
2,659472583,2022-05-28 08:01:24 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,429,0,False,False,commons.wikimedia.org
3,659475427,2022-05-28 08:20:47 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,4,0,False,False,commons.wikimedia.org
4,659476008,2022-05-28 08:25:16 UTC,https://commons.wikimedia.org,File:Residential Housing at Uganda Christian U...,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,63,0,False,False,commons.wikimedia.org


In [37]:
cd_wikis = pd.read_csv('https://raw.githubusercontent.com/wikimedia-research/canonical-data/master/wiki/wikis.tsv', sep='\t')
cd_wikis.head()

,database_code,domain_name,database_group,language_code,language_name,status,visibility,editability,english_name
0,aawiki,aa.wikipedia.org,wikipedia,aa,Afar,closed,public,public,Afar Wikipedia
1,aawikibooks,aa.wikibooks.org,wikibooks,aa,Afar,closed,public,public,Afar Wikibooks
2,aawiktionary,aa.wiktionary.org,wiktionary,aa,Afar,closed,public,public,Afar Wiktionary
3,abwiki,ab.wikipedia.org,wikipedia,ab,Abkhazian,open,public,public,Abkhazian Wikipedia
4,abwiktionary,ab.wiktionary.org,wiktionary,ab,Abkhazian,closed,public,public,Abkhazian Wiktionary


In [44]:
df = pd.merge(df, cd_wikis[['domain_name', 'database_code']], on='domain_name', how='left')
df.head()

,revision_id,timestamp,wiki,article_title,diff,username,bytes_added,references_added,new_article,dashboard_edit,domain_name,database_code
0,659472428,2022-05-28 08:00:24 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,402,0,True,False,commons.wikimedia.org,commonswiki
1,659472431,2022-05-28 08:00:25 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,240,0,False,False,commons.wikimedia.org,commonswiki
2,659472583,2022-05-28 08:01:24 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,429,0,False,False,commons.wikimedia.org,commonswiki
3,659475427,2022-05-28 08:20:47 UTC,https://commons.wikimedia.org,File:Ostrich at Entebbe Zoo.jpg,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,4,0,False,False,commons.wikimedia.org,commonswiki
4,659476008,2022-05-28 08:25:16 UTC,https://commons.wikimedia.org,File:Residential Housing at Uganda Christian U...,https://commons.wikimedia.org/w/index.php?titl...,Kateregga1,63,0,False,False,commons.wikimedia.org,commonswiki


In [45]:
wiki_dbs = df.database_code.unique().tolist()
wiki_dbs

['commonswiki',
 'frwiki',
 'enwiki',
 'incubatorwiki',
 'wikidatawiki',
 'twwiki',
 'hawiki',
 'rwwiki',
 'enwikivoyage',
 'igwiki',
 'yowiki',
 'enwikibooks',
 'enwikiquote']

In [28]:
users = df.username.unique().tolist()

In [30]:
# covert to Python list to sql tuple
sql_users = sql_tuple(users)

In [46]:
query = f"""
WITH reg_users AS 
	(SELECT
      user_registration,
      user_editcount,
      user_name    
    FROM
        user
    WHERE
        user_name IN {sql_users} AND
        YEAR(user_registration) = 2022 AND
   		MONTH(user_registration) = 5),
        
   revision AS (
     SELECT
     COUNT(rev_id),
     	actor_name
     FROM
     	revision
     	JOIN actor ON rev_actor = actor_id
     	JOIN reg_users ON reg_users.user_name = actor.actor_name
     WHERE
     	rev_timestamp > 20220501235959 AND
     	rev_timestamp < 20230101235959
--     YEAR(rev_timestamp) = 2022 AND
--     MONTH(rev_timestamp) > 5 AND
--     MONTH(rev_timestamp) <= 12
        GROUP BY actor_name

   )
     	
        
SELECT *
FROM revision
"""

result = query_wikis(query, wiki_dbs)
result

,COUNT(rev_id),actor_name,wiki_db
0,1,Efoby,commonswiki
1,5,Ishaqyusuf,commonswiki
2,3,MaskMan,commonswiki
3,4,Moctar Kamakaté,commonswiki
4,51,Nwonwu Uchechukwu P,commonswiki
5,9,Sadammuhammad11234,commonswiki
6,28,Ugoji.john,commonswiki
7,6,Yaseyda,commonswiki
0,1,Ibjaja055,frwiki
1,60,Yaseyda,frwiki


In [17]:
result.to_csv('result_set.csv')

### Misc code

In [ ]:
# with conn.cursor() as cur:
#     cur.execute(query)
#     data = cur.fetchall()
# conn.close()
# return data